In [12]:
import os
import tensorflow as tf
from PIL import Image
import numpy as np

# Define the folder location
folder_location = r"C:\Users\shaif\Downloads\Compressed\Labeled Baylor 8-30-23\Labeled Baylor 8-30-23\Zebra Umbo 1a Image1a\Umbonal"

folders = os.listdir(folder_location)

# Define the desired image size
target_size = (28, 28)

# Initialize an empty list to store the processed images
processed_images = []

# Define the number of images to read from each subdirectory
num_images_per_subdirectory = 5

# Iterate through each folder
for folder in folders:
    # Construct the folder path
    folder_path = os.path.join(folder_location, folder)
    
    # List all images in the folder
    images = os.listdir(folder_path)
    
    # Initialize a list to store images from this subdirectory
    subdirectory_images = []
    
    # Iterate through each image, up to the specified limit
    for i in range(min(num_images_per_subdirectory, len(images))):
        # Construct the image path
        image_path = os.path.join(folder_path, images[i])
        
        # Open the image using PIL
        image = Image.open(image_path)
        
        # Resize the image using TensorFlow
        resized_image = tf.image.resize_with_crop_or_pad(
            tf.keras.preprocessing.image.img_to_array(image),
            target_size[0],
            target_size[1]
        )
        
        # Normalize the data for deep learning
        normalized_image = (resized_image - 127.5) / 127.5
        
        # Append the processed image to the subdirectory list
        subdirectory_images.append(normalized_image)
    
    # Convert the list of images for this subdirectory to a NumPy array
    subdirectory_images = np.array(subdirectory_images)
    
    # Append the subdirectory images to the main list
    processed_images.append(subdirectory_images)

# Convert the list of images to a NumPy array
processed_images = np.array(processed_images)
processed_images.shape

(83, 5, 28, 28, 3)

In [10]:
import tensorflow as tf
from tensorflow.keras.models import load_model

#Patch Encoder with Conv2D ,  LSTM , Pos_Emd
from keras import layers

class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim, **kwargs):
        super(PatchEncoder, self).__init__(**kwargs)
        self.num_patches = num_patches
        self.projection_dim = projection_dim
        self.projection =keras.Sequential(
            [
                (layers.Conv2D(2, (3, 3), strides=(1,1),activation='relu')),
                TimeDistributed(MaxPooling2D(2,2)),
                TimeDistributed(Flatten()),
                layers.Dense(projection_dim)
            ]) 
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )
        
    def get_config(self):
        config = super().get_config()
        config.update({
            "num_patches": self.num_patches,
            "projection_dim": self.projection_dim,
        })
        return config

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded 


import keras
import tensorflow as tf
from keras.layers import TimeDistributed, Conv2D, Dense, MaxPooling2D, Flatten, LSTM, Dropout, BatchNormalization
from tensorflow.keras.models import load_model
model_path = r'E:\Video_Model\Video_Model\five_frame_model.h5'
# Load the saved Keras model with the custom metric
model = load_model(model_path, custom_objects={"PatchEncoder": PatchEncoder})

# Print the model summary
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 5, 28, 28,   0           []                               
                                3)]                                                               
                                                                                                  
 patch_encoder (PatchEncoder)   (None, 5, 32)        11064       ['input_1[0][0]']                
                                                                                                  
 layer_normalization (LayerNorm  (None, 5, 32)       64          ['patch_encoder[0][0]']          
 alization)                                                                                       
                                                                                              

In [38]:
x = model.predict(processed_images)
result = []

for i in x:
    arr = []
    o = i[0] - .4999
    n = i[1] + .4999
    m = i[2]
    
    arr.append(o)
    arr.append(n)
    arr.append(m)
    
    result.append(arr)

res = np.argmax(result,axis = 1)
count_0 = 0
count_1 = 0
count_2 = 0
# Count the numbers
for i in res:
    if(i==0):
        count_0  = count_0  + 1
    elif(i==1):
        count_1  = count_1  + 1
    else:
        count_2  = count_2  + 1

# Print the counts
print("Count of 0:", count_0)
print("Count of 1:", count_1)
print("Count of 2:", count_2)

Count of 0: 539
Count of 1: 443
Count of 2: 0


In [14]:
x = model.predict(processed_images)
res = np.argmax(x,axis = 1)
count_0 = 0
count_1 = 0
count_2 = 0
# Count the numbers
for i in res:
    if(i==0):
        count_0  = count_0  + 1
    elif(i==1):
        count_1  = count_1  + 1
    else:
        count_2  = count_2  + 1

# Print the counts
print("Count of 0:", count_0)
print("Count of 1:", count_1)
print("Count of 2:", count_2)

Count of 0: 73
Count of 1: 8
Count of 2: 2
